In [6]:
import psycopg2

In [7]:
from pathlib import Path

In [ ]:
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

# Create role

In [ ]:
# low risk secrets:
connection = psycopg2.connect(
                                  host    =  '127.0.0.1',
                                  user    =  'postgres',
                                  database=  'postgres',
                                  password=  "DaVinci",
                                  port    = "5432",
                                  connect_timeout=3,
                                )

In [ ]:
connection

In [ ]:
crs = connection.cursor()

In [ ]:
crs.execute("SELECT rolname FROM pg_roles")
crs.fetchall()

In [ ]:
username = ""
password = ""

psql = f"""
CREATE ROLE {username}
CREATEDB 
LOGIN 
PASSWORD '{password}';"""

print(psql)
try:
    crs.execute(psql)
except Exception as e:
    print(e)

In [ ]:
crs.execute("SELECT rolname FROM pg_roles")
crs.fetchall()

In [ ]:
# COMMIT CHANGES!
connection.commit()

In [ ]:
# CLOSE CURSOR
crs.close()

In [ ]:
# CLOSE CONNECTION
connection.close()

In [ ]:
connection

# Create Database

In [ ]:
# low risk secrets:
connection = psycopg2.connect(
                                  host    =  '127.0.0.1',
                                  user    =  'postgres',
                                  database=  'postgres',
                                  password=  "DaVinci",
                                  port    = "5432",
                                  connect_timeout=3,
                                )
connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

In [ ]:
cursor = connection.cursor()

In [ ]:
name = "garden"
user = "human"

cursor.execute(f"CREATE DATABASE {name} WITH OWNER={user}")

In [ ]:
# COMMIT CHANGES!
connection.commit()

In [ ]:
cursor.close()

In [ ]:
connection.close()

# Restart Server

In [11]:
with psycopg2.connect(
                    # the `postgres` db comes standard issue
                    database="postgres",
                    user="postgres",
                    password="DaVinci",
                    host="127.0.0.1",
                    port="5432",
                    connect_timeout=3,
                  ) as connection:

        crs = connection.cursor()

        # Find where postgres is installed (datadir and pg_ctl in bin)
        crs.execute("select name, setting from pg_settings where name = 'data_directory';")
        data_dir = Path(crs.fetchall()[0][1])
        pg_ctl = data_dir.absolute().parent / 'bin/pg_ctl'

        command = f'''cd {data_dir.absolute().parent}; sudo su postgres -c "{pg_ctl} restart -D {data_dir}"'''
        print("... restarting postgres >>>", command)
        
        crs.close()

... restarting postgres >>> cd /Library/PostgreSQL/9.5; sudo su postgres -c "/Library/PostgreSQL/9.5/bin/pg_ctl restart -D /Library/PostgreSQL/9.5/data"
